In [9]:
import pandas as pd
from typing import List, Dict


In [10]:
class DrugDatabase:
    def __init__(self, csv_path: str):
        self.interactions = {}
        self.load_dataset(csv_path)

    def load_dataset(self, csv_path: str):
        df = pd.read_csv(csv_path)
        for _, row in df.iterrows():
            d1 = self.standardize_name(row['Drug1'])
            d2 = self.standardize_name(row['Drug2'])
            key = tuple(sorted((d1, d2)))
            self.interactions[key] = {
                'interaction_type': row.get('interaction_type', ''),
                'description': row.get('description', 'No description.'),
                'severity': row.get('severity', 'unknown').lower()
            }

    def standardize_name(self, name: str) -> str:
        return name.strip().lower()

    def get_interaction(self, Drug1: str, Drug2: str) -> Dict:
        key = tuple(sorted((self.standardize_name(Drug1), self.standardize_name(Drug2))))
        return self.interactions.get(key, None)


In [11]:
class InteractionChecker:
    def __init__(self, db: DrugDatabase):
        self.db = db

    def check(self, medications: List[str]) -> List[Dict]:
        standardized = [self.db.standardize_name(med) for med in medications]
        interactions_found = []

        for i in range(len(standardized)):
            for j in range(i + 1, len(standardized)):
                interaction = self.db.get_interaction(standardized[i], standardized[j])
                if interaction:
                    interactions_found.append({
                        'Drug1': medications[i],
                        'Drug2': medications[j],
                        'severity': interaction['severity'],
                        'description': interaction['description']
                    })
        return interactions_found


In [12]:
class AlertSystem:
    def __init__(self):
        self.severity_rank = {'high': 1, 'moderate': 2, 'low': 3, 'unknown': 4}

    def sort_alerts(self, interactions: List[Dict]) -> List[Dict]:
        return sorted(interactions, key=lambda x: self.severity_rank.get(x['severity'], 4))

    def display(self, interactions: List[Dict]):
        if not interactions:
            print(" No significant drug interactions found.")
            return
        print(" Interaction Warnings:")
        for inter in interactions:
            print(f"[{inter['severity'].upper()}] {inter['Drug1']} + {inter['Drug2']} → {inter['description']}")


In [13]:

csv_path = r"C:\med_interaction_checker\Drug-Interaction\db_drug_interactions.csv"  

db = DrugDatabase(csv_path)
checker = InteractionChecker(db)
alerter = AlertSystem()

patient_meds =patient_meds = [
    "Aspirin",         
    "Warfarin",       
    "Lisinopril",      
    "Ibuprofen",       
    "Amiodarone",      
    "Metformin",      
    "Simvastatin",    
    "Clarithromycin"  
] 


print(f"\n Checking medications: {', '.join(patient_meds)}\n")
interactions = checker.check(patient_meds)
sorted_alerts = alerter.sort_alerts(interactions)
alerter.display(sorted_alerts)



 Checking medications: Aspirin, Warfarin, Lisinopril, Ibuprofen, Amiodarone, Metformin, Simvastatin, Clarithromycin

 Interaction Warnings:
[UNKNOWN] Warfarin + Ibuprofen → Warfarin may increase the anticoagulant activities of Ibuprofen.
[UNKNOWN] Warfarin + Amiodarone → Warfarin may increase the anticoagulant activities of Amiodarone.
[UNKNOWN] Warfarin + Simvastatin → Warfarin may increase the anticoagulant activities of Simvastatin.
[UNKNOWN] Warfarin + Clarithromycin → The metabolism of Clarithromycin can be decreased when combined with Warfarin.
[UNKNOWN] Lisinopril + Ibuprofen → The risk or severity of adverse effects can be increased when Lisinopril is combined with Ibuprofen.
[UNKNOWN] Lisinopril + Amiodarone → The risk or severity of adverse effects can be increased when Amiodarone is combined with Lisinopril.
[UNKNOWN] Lisinopril + Simvastatin → The serum concentration of Lisinopril can be increased when it is combined with Simvastatin.
[UNKNOWN] Lisinopril + Clarithromycin 

In [14]:

available_drugs = sorted(set(db.interactions.keys()))
flat_drug_list = sorted(set([drug for pair in available_drugs for drug in pair]))


print(" Available medications in the database:")
print(', '.join(flat_drug_list))


user_input = input("\nPlease enter combination from the above (comma-separated): ")
med_list = [m.strip() for m in user_input.split(',')]


print(f"\n Checking medications: {', '.join(med_list)}\n")
interactions = checker.check(med_list)
sorted_alerts = alerter.sort_alerts(interactions)
alerter.display(sorted_alerts)


 Available medications in the database:
abacavir, abemaciclib, abiraterone, acarbose, acebutolol, aceclofenac, acemetacin, acenocoumarol, acepromazine, aceprometazine, acetaminophen, acetazolamide, acetohydroxamic acid, acetophenazine, acetylcholine, acetyldigitoxin, acetylsalicylic acid, acipimox, acitretin, aclidinium, acrivastine, activated charcoal, acyclovir, adapalene, adefovir dipivoxil, adenosine, adinazolam, afatinib, agomelatine, ajmaline, albendazole, alcaftadine, alclofenac, alclometasone, alectinib, alendronic acid, alfacalcidol, alfentanil, alfuzosin, alimemazine, aliskiren, alitretinoin, allopurinol, allylestrenol, almasilate, almotriptan, alogliptin, alosetron, alprazolam, alprenolol, alprostadil, altretamine, aluminium, aluminum hydroxide, aluminum oxide, aluminum sulfate, alvimopan, amantadine, ambenonium, ambrisentan, ambroxol, amcinonide, amifostine, amikacin, amiloride, aminocaproic acid, aminoglutethimide, aminohippuric acid, aminolevulinic acid, aminophenazone, a


Please enter combination from the above (comma-separated):  Amiodarone, Warfarin, Clarithromycin



 Checking medications: Amiodarone, Warfarin, Clarithromycin

 Interaction Warnings:
[UNKNOWN] Amiodarone + Warfarin → Warfarin may increase the anticoagulant activities of Amiodarone.
[UNKNOWN] Amiodarone + Clarithromycin → The metabolism of Clarithromycin can be decreased when combined with Amiodarone.
[UNKNOWN] Warfarin + Clarithromycin → The metabolism of Clarithromycin can be decreased when combined with Warfarin.
